In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from keras import layers
from keras.models import Sequential

EPOCHS= 30
act= "swish"
opt= tf.keras.optimizers.Adam(learning_rate=0.001)
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import FastText
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, Dropout, Flatten, Bidirectional, Conv1D, concatenate, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling2D, GlobalAveragePooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping

In [ ]:
path = "/content/drive/MyDrive/Datasets/g5words-5authors.csv"

### Load Dataset

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

TEXT  ... word_count
0  रणधुरंधर, परमप्रतापी महाराणा प्रतापसिंह यांची ...  ...         12
1  मुंबईतील नेव्हल डॉकयार्ड येथे #INS Ranvir या य...  ...         37
2  जुन्नर तालुक्यातील शिंदेवाडीचे वीर सुपुत्र जवा...  ...         39
3  राज्याच्या सत्तर वर्षाच्या सामाजिक, राजकीय जीव...  ...         17
4  सर्वसामान्यांच्या प्रश्नावर रस्त्यावर उतरून लढ...  ...         30

[5 rows x 6 columns]

In [ ]:
df.USER.value_counts()

RRPSpeaks          2923
kolhe_amol         2780
PawarSpeaks        2493
rajeshtope11       2400
uddhavthackeray    2320
Name: USER, dtype: int64

In [ ]:
df.word_count.unique()

array([12, 37, 39, 17, 30, 36, 26,  9, 28, 35, 18, 25, 21, 33, 15, 22, 32,
       34, 38,  6, 40, 29, 14, 24, 41, 31, 23, 19, 20, 13, 27, 16, 44, 11,
       10, 43, 48,  8,  7, 42, 45, 46, 47, 51, 49])

In [ ]:
Y_train = df['USER']
Y_train.head()

0    kolhe_amol
1    kolhe_amol
2    kolhe_amol
3    kolhe_amol
4    kolhe_amol
Name: USER, dtype: object

### User Encoding

In [ ]:
df['label'] = le.fit_transform(Y_train)

In [ ]:
df[['USER', 'label']].drop_duplicates(keep='first')

USER  label
0           kolhe_amol      2
2780       PawarSpeaks      0
5273      rajeshtope11      3
7673         RRPSpeaks      1
10596  uddhavthackeray      4

In [ ]:
X_train = df['clean_text']
Y_train = df['label']

In [ ]:
sentences_ted = []

In [ ]:
X_train = X_train.astype(str)

### Created vocabulary

In [ ]:
for sent_str in X_train:
  tokens = sent_str.lower().split()
  sentences_ted.append(tokens)

### Tokenization

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

### Splitting dataset into train-test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size

48917

In [ ]:
lens_train = [len(i) for i in X_train]
lens_test = [len(i) for i in X_test]
lens = lens_train + lens_test

maxlen = np.max(lens)
print('Max len:', maxlen)

Max len: 51


In [ ]:
s = pd.Series(lens)
s.value_counts()

33    555
34    529
36    522
37    511
35    486
32    476
31    425
30    409
29    407
25    398
38    388
22    378
28    369
24    357
27    351
41    348
26    335
23    333
39    330
16    315
40    313
15    312
17    309
14    293
21    289
18    283
19    280
13    276
20    272
11    271
12    269
43    260
10    219
9     216
8     210
6     194
7     188
42    145
44     42
45     34
46      7
47      5
48      3
49      3
51      1
dtype: int64

### Padding
Made all the tweets of equal length, by padding zeros at the end.

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=43)
X_test = pad_sequences(X_test, padding='post', maxlen=43)

### Training FastText model for our vocabulary.

In [ ]:
model_ted = FastText(sentences_ted, size=300, window=5, min_count=5, workers=8,sg=1)

### Generating embedding matrix for each tweet.

In [ ]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, 300))
for word,i in word_index.items():
    try:
        embedding_vector = model_ted.wv[word]
    except:
        print(word, 'not found')
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

91 not found
भड not found
ई not found
७३ not found
४९ not found
४१ not found
सढळ not found
७२ not found
नॅशनल not found
६१ not found
84084 not found
९३ not found
नफा not found
६२ not found
ऐन not found
97 not found
39 not found
34 not found
81 not found
71 not found
59 not found
35 not found
85 not found
90 not found
ॲप not found
६६ not found
ओ not found
६४ not found
49 not found
ओघ not found
थॉमस not found
९७ not found
आए। not found
९२ not found
– not found
५८ not found
४३ not found
32 not found
360 not found
43 not found
07 not found
92 not found
73 not found
76 not found
62 not found
55 not found
9346 not found
60 not found
9249 not found
9496 not found
लुडबूड not found
८९ not found
लुई not found
उ not found
एफ not found
० not found
॥ not found
इद not found
अढळ not found
ी not found
९६ not found
र not found
४७ not found
ऑइल not found
68 not found
74 not found
38 not found
357 not found
94 not found
82 not found
77 not found
01 not found
042 not found
46 not found
05 not found
45 not

### CNN model preparation

In [ ]:
  myCNN = Sequential()
  myCNN.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=43))
  myCNN.add(Conv1D(filters = 32 , kernel_size=3,activation='relu'))
  myCNN.add(GlobalMaxPooling1D())
  myCNN.add(Dropout(0.3))
  myCNN.add(Dense(5, activation='softmax'))
  myCNN.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])

In [ ]:
myCNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 43, 300)           14675100  
                                                                 
 conv1d (Conv1D)             (None, 41, 32)            28832     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 14,704,097
Trainable params: 14,704,097
Non-trainable params: 0
____________________________________________

In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

### Training

In [ ]:
history = myCNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = myCNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myCNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
81/81 [==============================] - 12s 24ms/step - loss: 1.2973 - acc: 0.4717 - val_loss: 1.0400 - val_acc: 0.6463
Epoch 2/10
81/81 [==============================] - 2s 19ms/step - loss: 0.9559 - acc: 0.6485 - val_loss: 0.8532 - val_acc: 0.7152
Epoch 3/10
81/81 [==============================] - 2s 20ms/step - loss: 0.7335 - acc: 0.7545 - val_loss: 0.7374 - val_acc: 0.7508
Epoch 4/10
81/81 [==============================] - 2s 19ms/step - loss: 0.5446 - acc: 0.8303 - val_loss: 0.6613 - val_acc: 0.7709
Epoch 5/10
81/81 [==============================] - 2s 19ms/step - loss: 0.3963 - acc: 0.8879 - val_loss: 0.6075 - val_acc: 0.7875
Epoch 6/10
81/81 [==============================] - 2s 19ms/step - loss: 0.2878 - acc: 0.9242 - val_loss: 0.5690 - val_acc: 0.7972
Epoch 7/10
81/81 [==============================] - 2s 19ms/step - loss: 0.2093 - acc: 0.9482 - val_loss: 0.5480 - val_acc: 0.8038
Epoch 8/10
81/81 [==============================] - 2s 19ms/step - loss: 0.1618 - 

**BiLSTM**

In [ ]:
myLSTM = Sequential()
myLSTM.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=43))
myLSTM.add(layers.Bidirectional(layers.LSTM(256, recurrent_dropout=0.3,return_sequences=True)))
myLSTM.add(GlobalMaxPooling1D())
myLSTM.add(Dense(64, activation='relu'))
myLSTM.add(Dense(5, activation='softmax'))
myLSTM.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
myLSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 43, 300)           14675100  
                                                                 
 bidirectional (Bidirectiona  (None, 43, 512)          1140736   
 l)                                                              
                                                                 
 global_max_pooling1d_1 (Glo  (None, 512)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 15,848,993
Trainable params: 15,848,993


In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = myLSTM.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 


Epoch 1/10
81/81 [==============================] - 54s 594ms/step - loss: 0.9977 - acc: 0.6018 - val_loss: 0.7276 - val_acc: 0.7121
Epoch 2/10
81/81 [==============================] - 46s 570ms/step - loss: 0.5336 - acc: 0.8062 - val_loss: 0.5475 - val_acc: 0.7941
Epoch 3/10
81/81 [==============================] - 48s 592ms/step - loss: 0.2372 - acc: 0.9190 - val_loss: 0.4321 - val_acc: 0.8464
Epoch 4/10
81/81 [==============================] - 45s 557ms/step - loss: 0.0875 - acc: 0.9710 - val_loss: 0.5473 - val_acc: 0.8390
Epoch 5/10
81/81 [==============================] - 46s 566ms/step - loss: 0.0441 - acc: 0.9879 - val_loss: 0.5440 - val_acc: 0.8348
Epoch 6/10
81/81 [==============================] - 46s 564ms/step - loss: 0.0201 - acc: 0.9938 - val_loss: 0.6468 - val_acc: 0.8336
Epoch 7/10
81/81 [==============================] - 45s 559ms/step - loss: 0.0209 - acc: 0.9935 - val_loss: 0.7586 - val_acc: 0.8170
Epoch 8/10
81/81 [==============================] - 45s 553ms/step - 

In [ ]:
loss, accuracy = myLSTM.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myLSTM.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

323/323 [==============================] - 16s 51ms/step - loss: 0.0059 - acc: 0.9981
Training Accuracy: 0.9981
81/81 [==============================] - 4s 51ms/step - loss: 0.6022 - acc: 0.8557
Testing Accuracy:  0.8557


**BiLSTM + Attention**

In [ ]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, input_shape):
        self.W_in = self.add_weight(shape=(input_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
       
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                        initializer='zeros',
                                        trainable=True,
                                        name='attention_bias')
        
        super(AttentionLayer, self).build(input_shape)
        
        
    def call(self, lstm_output):
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = lstm_output * at
        return attention_output
    
    
    def compute_output_shape(self, input_state):
        return (input_state[0], input_state[-1])
    
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [ ]:
Input1 = Input(shape=(43,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.3, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
max_pool = MaxPooling1D(pool_size=5)(Attention)
Dropout1 = Dropout(0.3)(max_pool)
Flat = Flatten()(Dropout1)
Dense1 = Dense(512, activation='relu')(Flat)
Dropout2 = Dropout(0.3)(Dense1)
Dense2 = Dense(64, activation='relu')(Dropout2)
Dropout3 = Dropout(0.25)(Dense2)
outputs = Dense(5, activation='softmax')(Dropout3)

model = Model(inputs=Input1, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 43)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 43, 300)           14675100  
                                                                 
 bidirectional_1 (Bidirectio  (None, 43, 128)          186880    
 nal)                                                            
                                                                 
 attention_layer (AttentionL  (None, 43, 128)          171       
 ayer)                                                           
                                                                 
 max_pooling1d (MaxPooling1D  (None, 8, 128)           0         
 )                                                               
                                                             

In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
81/81 [==============================] - 9s 53ms/step - loss: 1.2254 - acc: 0.4859 - val_loss: 0.9297 - val_acc: 0.6447
Epoch 2/10
81/81 [==============================] - 3s 39ms/step - loss: 0.8140 - acc: 0.6951 - val_loss: 0.6950 - val_acc: 0.7423
Epoch 3/10
81/81 [==============================] - 3s 41ms/step - loss: 0.4730 - acc: 0.8333 - val_loss: 0.5559 - val_acc: 0.7953
Epoch 4/10
81/81 [==============================] - 3s 39ms/step - loss: 0.2316 - acc: 0.9214 - val_loss: 0.5709 - val_acc: 0.8146
Epoch 5/10
81/81 [==============================] - 3s 39ms/step - loss: 0.1252 - acc: 0.9611 - val_loss: 0.6723 - val_acc: 0.8073
Epoch 6/10
81/81 [==============================] - 3s 39ms/step - loss: 0.0737 - acc: 0.9782 - val_loss: 0.6250 - val_acc: 0.8286
Epoch 7/10
81/81 [==============================] - 3s 39ms/step - loss: 0.0517 - acc: 0.9843 - val_loss: 0.9896 - val_acc: 0.7678
Epoch 8/10
81/81 [==============================] - 3s 39ms/step - loss: 0.0441 - a

**Serial BiLSTM and CNN**

In [ ]:
Input1 = Input(shape=(43,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.25, return_sequences=True))(Embedding_layer1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu')(LSTM1)
max_pooling1 = MaxPooling1D(pool_size=3)(Conv1)
Dropout2 = Dropout(0.25)(max_pooling1)
Flatten1 = Flatten()(Dropout2)
Dense1 = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(Flatten1)
Dense2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(Dense1)
output = Dense(5, activation='softmax')(Dense2)

myBiLSTM_CNN = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 43)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 43, 300)           14675100  
                                                                 
 bidirectional_4 (Bidirectio  (None, 43, 128)          186880    
 nal)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 39, 64)            41024     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 13, 64)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 13, 64)            0   

In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = myBiLSTM_CNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = myBiLSTM_CNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
81/81 [==============================] - 3s 43ms/step - loss: 0.0451 - acc: 0.9935 - val_loss: 0.7805 - val_acc: 0.8100
Epoch 2/10
81/81 [==============================] - 3s 41ms/step - loss: 0.0406 - acc: 0.9946 - val_loss: 0.5832 - val_acc: 0.8351
Epoch 3/10
81/81 [==============================] - 3s 40ms/step - loss: 0.0355 - acc: 0.9958 - val_loss: 0.7061 - val_acc: 0.8301
Epoch 4/10
81/81 [==============================] - 3s 40ms/step - loss: 0.0304 - acc: 0.9959 - val_loss: 0.6989 - val_acc: 0.8378
Epoch 5/10
81/81 [==============================] - 3s 41ms/step - loss: 0.0290 - acc: 0.9962 - val_loss: 0.7080 - val_acc: 0.8235
Epoch 6/10
81/81 [==============================] - 3s 41ms/step - loss: 0.0274 - acc: 0.9965 - val_loss: 0.7737 - val_acc: 0.8220
Epoch 7/10
81/81 [==============================] - 3s 41ms/step - loss: 0.0292 - acc: 0.9966 - val_loss: 0.8490 - val_acc: 0.8251
Epoch 8/10
81/81 [==============================] - 3s 41ms/step - loss: 0.0286 - a

**BiLSTM and CNN serial with Attention**

In [ ]:
Input1 = Input(shape=(43,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

LSTM1 = Bidirectional(LSTM(128, dropout=0.25, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(Attention)
max_pool = GlobalMaxPooling1D()(Conv1)
Dropout2 = Dropout(0.25)(max_pool)
output = Dense(5, activation='softmax')(Dropout2)

myBiLSTM_CNN_Att = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN_Att.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN_Att.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 43)]              0         
                                                                 
 embedding_6 (Embedding)     (None, 43, 300)           14675100  
                                                                 
 bidirectional_5 (Bidirectio  (None, 43, 256)          439296    
 nal)                                                            
                                                                 
 attention_layer_1 (Attentio  (None, 43, 256)          299       
 nLayer)                                                         
                                                                 
 conv1d_4 (Conv1D)           (None, 43, 64)            81984     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0   

In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = myBiLSTM_CNN_Att.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback]) 
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
81/81 [==============================] - 5s 57ms/step - loss: 0.0167 - acc: 0.9949 - val_loss: 0.8777 - val_acc: 0.8111
Epoch 2/10
81/81 [==============================] - 4s 49ms/step - loss: 0.0076 - acc: 0.9975 - val_loss: 0.8835 - val_acc: 0.8220
Epoch 3/10
81/81 [==============================] - 4s 50ms/step - loss: 0.0078 - acc: 0.9970 - val_loss: 0.9545 - val_acc: 0.8204
Epoch 4/10
81/81 [==============================] - 4s 50ms/step - loss: 0.0070 - acc: 0.9973 - val_loss: 0.9033 - val_acc: 0.8142
Epoch 5/10
81/81 [==============================] - 4s 50ms/step - loss: 0.0084 - acc: 0.9973 - val_loss: 0.9187 - val_acc: 0.8015
Epoch 6/10
81/81 [==============================] - 4s 50ms/step - loss: 0.0071 - acc: 0.9973 - val_loss: 0.8657 - val_acc: 0.8162
Epoch 7/10
81/81 [==============================] - 4s 49ms/step - loss: 0.0057 - acc: 0.9981 - val_loss: 0.9037 - val_acc: 0.8228
Epoch 8/10
81/81 [==============================] - 4s 49ms/step - loss: 0.0048 - a

**parallel biLSTM and CNN with attention**

In [ ]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, attention_input):
        super(AttentionLayer, self).build(attention_input)
        lstm_shape, cnn_shape = attention_input
        # Attention Weights for LSTM
        self.W_in = self.add_weight(shape=(lstm_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
        
        # Attention Weights for CNN
        self.W_context = self.add_weight(shape=(cnn_shape[-1], 1),
                                         initializer='glorot_normal',
                                         trainable=True,
                                         name='context_attention_weights')
                                      
        # Attention Bias
        self.b = self.add_weight(shape=(lstm_shape[1], 1),
                                 initializer='glorot_normal',
                                 trainable=True,
                                 name='attention_bias')
        
        
        
    def call(self, attention_input):
        lstm_output, cnn_output = attention_input
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) + 
                              K.dot(cnn_output, self.W_context) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = at * lstm_output
        attention_output = K.sum(attention_output, axis=1)
        return attention_output
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [ ]:
Input1 = Input(shape=(43,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

# Input 1(BiLSTM):
Lstm = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(Embedding_layer1)

# Input 2 (CNN):
Conv1 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(Embedding_layer1)
Dropout1 = Dropout(0.3)(Conv1)
Conv2 = Conv1D(filters=256, kernel_size=4, activation='relu', padding='same')(Embedding_layer1)
Dropout2 = Dropout(0.3)(Conv2)
Conv3 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(Embedding_layer1)
Dropout3 = Dropout(0.3)(Conv3)
merged = concatenate([Dropout1, Dropout2, Dropout3], axis=1)
max_pool = MaxPooling1D(pool_size=3)(merged)

attention = AttentionLayer()([Lstm, max_pool])
Dense1 = Dense(64, activation='relu')(attention)
Dropout_dense = Dropout(0.25)(Dense1)
outputs = Dense(5, activation='softmax')(Dropout_dense)

classifier = Model(inputs=Input1, outputs=outputs)
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
classifier.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 43)]         0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, 43, 300)      14675100    ['input_6[0][0]']                
                                                                                                  
 conv1d_5 (Conv1D)              (None, 43, 256)      230656      ['embedding_7[0][0]']            
                                                                                                  
 conv1d_6 (Conv1D)              (None, 43, 256)      307456      ['embedding_7[0][0]']            
                                                                                            

In [ ]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
ec = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
reduce_lr_callback = ReduceLROnPlateau( monitor='val_acc', 
                                       factor=0.1, 
                                       patience=2, 
                                       min_lr=0.0005, 
                                       verbose=2)

In [ ]:
history = classifier.fit(X_train, y_train, epochs=16, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = classifier.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = classifier.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/16
81/81 [==============================] - 7s 85ms/step - loss: 0.0123 - acc: 0.9962 - val_loss: 0.7681 - val_acc: 0.8375 - lr: 5.0000e-04
Epoch 2/16
81/81 [==============================] - 7s 83ms/step - loss: 0.0090 - acc: 0.9967 - val_loss: 0.7523 - val_acc: 0.8406 - lr: 5.0000e-04
Epoch 3/16
81/81 [==============================] - 6s 79ms/step - loss: 0.0086 - acc: 0.9969 - val_loss: 0.7699 - val_acc: 0.8375 - lr: 5.0000e-04
Epoch 4/16
81/81 [==============================] - 6s 79ms/step - loss: 0.0078 - acc: 0.9974 - val_loss: 0.7851 - val_acc: 0.8359 - lr: 5.0000e-04
Epoch 5/16
323/323 [==============================] - 5s 16ms/step - loss: 0.0032 - acc: 0.9983
Training Accuracy: 0.9983
81/81 [==============================] - 1s 17ms/step - loss: 0.7948 - acc: 0.8375
Testing Accuracy:  0.8375
